# Smart Resize

Resizes images without distortion, by applying cropping on the longer dimension before resizing.

The process is:
* Take the largest centered crop of the image that has the same aspect ratio as the target size. For instance, if size=(200, 200) and the input image has size (340, 500), we take a crop of (340, 340) centered along the width.
* Resize the cropped image to the target size. In the example above, we resize the (340, 340) crop to (200, 200).


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

In [ ]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [ ]:
train_files = getListOfFiles('../input/petfinder-pawpularity-score/train')
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv').set_index('Id')
train_ids = [os.path.basename(file).split('.')[0] for file in train_files]

test_files = getListOfFiles('../input/petfinder-pawpularity-score/test')
y_train = train_df.loc[train_ids].Pawpularity

In [ ]:
def read_resize(file, size):
    x = tf.io.read_file(file)
    x = tf.io.decode_jpeg(x)
    x = tf.keras.preprocessing.image.smart_resize(x, size, interpolation='bilinear')
    x = x / 256
    return x.numpy()

In [ ]:
x_train_256 = np.stack([read_resize(file, (256, 256)) for file in tqdm(train_files)])
x_train_256.nbytes / 1024 / 1024 / 1024

In [ ]:
os.makedirs('../working/petfinder-pawpularity-score-resized')

In [ ]:
np.save('../working/petfinder-pawpularity-score-resized/x_train_256.npy', x_train_256)
np.save('../working/petfinder-pawpularity-score-resized/y_train.npy', y_train)